In [ ]:
# Test with suspicious claim
test_claim = "Medical claim $45,000. Patient has 4 similar claims in 6 months. Out-of-network provider. Diagnosis codes don't match procedures."

print("="*80)
print("TESTING FRAUD DETECTION AGENT")
print("="*80)
print(f"Claim: {test_claim}")
print("\nAgent Analysis:")
print("="*80)

result = agent.invoke({"messages": [("user", f"Analyze this claim for fraud:\n{test_claim}")]})

# Print conversation
for msg in result['messages']:
    if hasattr(msg, 'content') and msg.content:
        print(f"\n{msg.type.upper()}: {msg.content[:500]}")

print("\n" + "="*80)
print("✅ AGENT TEST COMPLETE")
print("="*80)
